In [1]:
import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

Using TensorFlow backend.


In [2]:
max_features = 2000
max_len = 500

In [3]:
(x_train, y_train), (x_val, y_val) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_val = sequence.pad_sequences(x_val, maxlen=max_len)

In [4]:
model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128,
            input_length=max_len,
            name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
_________________________________________________________________


In [5]:
callbacks_list = [
    # Interrupts training when improvement stops
    keras.callbacks.EarlyStopping(
        monitor='acc', # Monitor model's validation accuracy
        patience=1, # Interrupt training when accuracy has stopped improving for more than 1 epochs
    ),
    # Save current weight after each epoch
    keras.callbacks.ModelCheckpoint(
        filepath='my_model.h5', # Path
        monitor='val_loss', # Save only if val loss is better than previous
        save_best_only=True,
    )
]

In [6]:
model.compile(optimizer='rmsprop',
                loss='binary_crossentropy',
                metrics=['acc'])

In [7]:
model.fit(x_train, y_train,
         epochs=10,
         batch_size=32,
         callbacks=callbacks_list,
         validation_data=(x_val, y_val))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 17s 664us/step - loss: 0.5748 - acc: 0.7301 - val_loss: 0.4392 - val_acc: 0.8101
Epoch 2/10
25000/25000 [==============================] - 15s 617us/step - loss: 0.4743 - acc: 0.7386 - val_loss: 0.4882 - val_acc: 0.7256
Epoch 3/10
25000/25000 [==============================] - 16s 642us/step - loss: 0.4590 - acc: 0.6383 - val_loss: 0.5283 - val_acc: 0.6350


In [8]:
!ls

'Keras Callbacks.ipynb'   my_model.h5


In [9]:
callbacks_list = [
    # Reduce the LR
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', # Monitors the model's validation loss
        factor=0.1, # Divide LR by 10 times when triggred
        patience=10, # callback is triggered after the validation loss has stopped improving for 10 epochs.
    )
]

In [10]:
model.fit(x_train, y_train,
         epochs=10,
         batch_size=32,
         callbacks=callbacks_list,
         validation_data=(x_val, y_val))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 14s 577us/step - loss: 0.3729 - acc: 0.5742 - val_loss: 0.6348 - val_acc: 0.5605
Epoch 2/10
25000/25000 [==============================] - 15s 583us/step - loss: 0.3169 - acc: 0.4904 - val_loss: 0.8422 - val_acc: 0.4199
Epoch 3/10
25000/25000 [==============================] - 15s 589us/step - loss: 0.2641 - acc: 0.4008 - val_loss: 0.9213 - val_acc: 0.3482
Epoch 4/10
25000/25000 [==============================] - 15s 597us/step - loss: 0.2308 - acc: 0.2924 - val_loss: 1.2992 - val_acc: 0.2993
Epoch 5/10
25000/25000 [==============================] - 15s 593us/step - loss: 0.2156 - acc: 0.1989 - val_loss: 1.3376 - val_acc: 0.2174
Epoch 6/10
25000/25000 [==============================] - 15s 607us/step - loss: 0.2086 - acc: 0.1632 - val_loss: 1.3850 - val_acc: 0.1860
Epoch 7/10
25000/25000 [==============================] - 15s 617us/step - loss: 0.2028 - acc: 0.1182 - val_loss: 1.

Example of a custom callback that saves to disk (as Numpy arrays) the
activations of every layer of the model at the end of every epoch, computed on the
first sample of the validation set:

In [42]:
import numpy as np

class ActivationLogger(keras.callbacks.Callback):
    
    def set_model(self, model):
        self.model = model
        layer_output = [layer.output for layer in model.layers]
        self.activations_model = keras.models.Model(model.input,
                                                   layer_output)
        
    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError('Requires validation_data.')
        validation_sample = self.validation_data[0][0:1]
        activations = self.activations_model.predict(validation_sample)
        f = open('activations_at_epoch_' + str(epoch) + '.npz', 'wb')
        np.savez(f, *activations)
        f.close()

In [43]:
callbacks_list = [ActivationLogger()]

In [45]:
model.fit(x_train, y_train,
         epochs=5,
         batch_size=32,
         callbacks=callbacks_list,
         validation_data=(x_val, y_val))

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 24s 965us/step - loss: 0.2041 - acc: 0.0186 - val_loss: 1.7907 - val_acc: 0.0647
Epoch 2/5
25000/25000 [==============================] - 21s 835us/step - loss: 0.2045 - acc: 0.0172 - val_loss: 1.8560 - val_acc: 0.0647
Epoch 3/5
25000/25000 [==============================] - 32s 1ms/step - loss: 0.1915 - acc: 0.0186 - val_loss: 1.8072 - val_acc: 0.0592
Epoch 4/5
25000/25000 [==============================] - 28s 1ms/step - loss: 0.1922 - acc: 0.0168 - val_loss: 1.8205 - val_acc: 0.0573
Epoch 5/5
25000/25000 [==============================] - 27s 1ms/step - loss: 0.1930 - acc: 0.0171 - val_loss: 1.8526 - val_acc: 0.0574


In [46]:
!ls

 activations_at_epoch_0.npz   activations_at_epoch_3.npz   my_model.h5
 activations_at_epoch_1.npz   activations_at_epoch_4.npz
 activations_at_epoch_2.npz  'Keras Callbacks.ipynb'


In [51]:
activations_at_epoch_0 = np.load('activations_at_epoch_0.npz')
print(activations_at_epoch_0['arr_0'])

[[[-0.05784114  0.03194386  0.06401597 ...  0.00056683 -0.04904091
   -0.03265695]
  [-0.05784114  0.03194386  0.06401597 ...  0.00056683 -0.04904091
   -0.03265695]
  [-0.05784114  0.03194386  0.06401597 ...  0.00056683 -0.04904091
   -0.03265695]
  ...
  [-0.02606118  0.11000165  0.00628612 ... -0.058791   -0.02768152
    0.00669897]
  [ 0.10527123 -0.01816737  0.01716958 ...  0.03317714  0.03957639
    0.03021466]
  [ 0.03561759 -0.12553193  0.01477517 ...  0.08704042  0.01571502
   -0.1405469 ]]]
